In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil
import glob
import time
import tensorflow as tf
import sklearn
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir='../input/100-bird-species/train'
valid_dir='../input/100-bird-species/valid'
test_dir='../input/100-bird-species/test'

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.2, preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input, horizontal_flip=True, rotation_range=0.2)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)

In [ ]:
target_size=(256,256)
batch_size=32

In [ ]:
train_set = train_datagen.flow_from_directory(train_dir, target_size=target_size, batch_size=batch_size, class_mode='categorical', shuffle=True)
valid_set = test_datagen.flow_from_directory(valid_dir, target_size=target_size, batch_size=batch_size, class_mode='categorical', shuffle=False)
test_set = test_datagen.flow_from_directory(test_dir, target_size=target_size, batch_size=batch_size, class_mode='categorical', shuffle=True)
test_x, test_y = next(test_set)

In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(include_top=False, input_shape=(256,256,3))

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.Sequential([
    base_model, tf.keras.layers.GlobalAveragePooling2D(), tf.keras.layers.Dense(265, activation='softmax')
])
lr=0.001
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])

In [ ]:
patience = 1
stop_patience = 3
factor = 0.5

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=stop_patience, monitor='val_loss', verbose=1, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience, verbose=1)
]

In [ ]:
epochs = 15
history = model.fit(train_set, validation_data=valid_set, epochs=epochs, callbacks=callbacks, verbose=1)

In [ ]:
model.evaluate(test_set)